# Analyzing FHIR data in Databricks

In [2]:
# Some initial setting:
storageAccountName = "<mystorageaccountName>"
storageAccountKey = "<mystorageaccountKey>"

In [3]:
dbutils.fs.mount(
  source = "wasbs://dataexport@" + storageAccountName + ".blob.core.windows.net",
  mount_point = '/mnt/dataexport',
  extra_configs = {"fs.azure.account.key." + storageAccountName + ".blob.core.windows.net":storageAccountKey})

Out[ 7 ]: True

In [4]:
%sql CREATE TEMPORARY TABLE observationTable USING json OPTIONS (path "/mnt/dataexport/Observation.json")

In [5]:
%sql CREATE TEMPORARY TABLE patientTable USING json OPTIONS (path "/mnt/dataexport/Patient.json")

In [6]:
%sql CREATE OR REPLACE TEMPORARY VIEW temp_heights AS SELECT * FROM (SELECT subject.reference as patient, valueQuantity.value as heightValue, valueQuantity.unit as heightUnit,  ROW_NUMBER() OVER (PARTITION BY subject.reference ORDER BY issued DESC) AS rn FROM observationTable WHERE code.coding[0].code = "8302-2") tm WHERE tm.rn = 1

In [7]:
%sql CREATE OR REPLACE TEMPORARY VIEW temp_weights AS SELECT * FROM (SELECT subject.reference as patient, valueQuantity.value as weightValue, valueQuantity.unit as weightUnit,  ROW_NUMBER() OVER (PARTITION BY subject.reference ORDER BY issued DESC) AS rn FROM observationTable WHERE code.coding[0].code = "29463-7") tm WHERE tm.rn = 1

In [8]:
%sql 
CREATE OR REPLACE TEMPORARY VIEW temp_latitude AS SELECT id, coord.valueDecimal AS latitude FROM (SELECT id, explode(address[0].extension[0].extension) as coord FROM patientTable) WHERE coord.url = 'latitude';
CREATE OR REPLACE TEMPORARY VIEW temp_longitude AS SELECT id, coord.valueDecimal AS longitude FROM (SELECT id, explode(address[0].extension[0].extension) as coord FROM patientTable) WHERE coord.url = 'longitude'

In [9]:
%sql SELECT patientTable.id, patientTable.name[0].family AS lastName, temp_longitude.longitude AS longitude, temp_latitude.latitude AS latitude, temp_weights.weightValue, temp_heights.heightValue FROM patientTable INNER JOIN temp_weights ON temp_weights.patient LIKE concat('%', patientTable.id, '%') INNER JOIN temp_heights ON temp_heights.patient LIKE concat('%', patientTable.id, '%') INNER JOIN temp_latitude ON temp_latitude.id = patientTable.id INNER JOIN temp_longitude ON temp_longitude.id = patientTable.id

id,lastName,longitude,latitude,weightValue,heightValue
0f7c47c3-5590-4b31-9bb7-5c63ce7ac2a7,Mohr916,-70.599243,41.594939,92.13042895893534,192.85297269012312
8aee0b6e-21fb-4f6c-aae9-2d83e532e17d,Lakin515,-71.45625600000002,42.425955,80.15757577973014,190.54977969241796
5c5e4974-7d28-4d98-84d4-18eb3eb4f788,Krajcik437,-71.24226800000002,42.388921,13.3112797373299,81.28613786091779
d01e01e5-2674-45c6-b69c-ecf626c974a7,Rowe323,-70.969734,42.534202,16.595222814972455,96.0173701365811
f37fa7c5-aedf-4e86-a325-9d36a6975367,Hahn503,-71.142002,42.323853,84.36838963519705,180.61652820180467
41b23605-b4a8-4526-bec5-ece177b95d12,Tromp100,-71.282917,42.305932,115.27609005869526,166.61076868883652
c3a470cd-b69e-4565-8f1c-2f0c85c626d9,Griego535,-71.020173,42.33196,81.06196431648411,160.23362754471327
d82e0e5c-f144-4968-a0d2-ad702436e450,Kassulke119,-71.177925,42.369451,122.62057850953231,173.76949774678
b9f20a15-b3eb-49f5-bf60-fde29e82ee96,Hyatt152,-70.62403,41.897892,119.83206894262251,178.1054736153097
a6e14901-9fd2-4836-9332-7d1d5d74bd9e,Heller342,-72.539978,42.115454,109.23505819790152,177.46609357404847
